In [2]:
# Data wrangling
import pandas as pd

# Scientific
import numpy as np

# Machine learning
## After installing the environment, remove xgboost and libxgboost from environment and install xgboost==1.6.2
from xgboost import XGBRegressor, Booster

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance
from sklearn import preprocessing
    
import shap
    
# Graphics
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns # for correlation heatmap

#model loader
import pickle

In [6]:
seed=100

filename = '../data/pH_prediction/Ramoneda2023_pH_predictor_tool/pH_model_2022-10-17_56_full.sav'
model_perm_subset = pickle.load(open(filename, 'rb'))
df = pd.read_csv('../data/pH_prediction/Ramoneda2023_pH_predictor_tool/56_genes_pHpreference_soil_freshwater.csv')#TABLE THAT INCLUDES PH PREFERENCES AND THE PRESENCE/ABSENCE OF THE 56 GENES ASSOCIATED TO PH PREFERENCE

In [7]:
gene_order = ['pH.preference','X5.FTHF_cyc.lig','AAA_25','AAA_assoc_C','AcetylCoA_hyd_C','ASH','Big_3_5','CitMHS','CpsB_CapC','CpXC','CsbD','Cys_rich_CPXG','Cytidylate_kin2','CytoC_RC',
              'DHquinase_I','Exo_endo_phos','FAD_binding_7','Glucodextran_N','GWxTD_dom','HipA_2','HTH_37','HupF_HypC','HycI','HypD','Ig_GlcNase','KdpA','KdpC','KdpD','Lactonase','MCRA','Met_gamma_lyase',
              'Methyltransf_4','MgtE','MNHE','MrpF_PhaF','OprB','Paired_CXXCH_1','PDZ_2','PGI','PhaG_MnhG_YufB','Phenol_MetA_deg','Phosphoesterase','Polbeta','PQQ','Pro.kuma_activ','SelO','SNase',
              'SOUL','TctA','TelA','TFR_dimer','TPP_enzyme_C','TrbI','UvdE','WXXGXW','YHS','zf.CDGSH']

df = df.reindex(columns=gene_order)
df

,pH.preference,X5.FTHF_cyc.lig,AAA_25,AAA_assoc_C,AcetylCoA_hyd_C,ASH,Big_3_5,CitMHS,CpsB_CapC,CpXC,...,SOUL,TctA,TelA,TFR_dimer,TPP_enzyme_C,TrbI,UvdE,WXXGXW,YHS,zf.CDGSH
0,3.724741,1,1,1,1,0,0,1,0,0,...,1,0,0,0,1,1,0,1,1,1
1,3.839540,1,1,1,1,0,0,1,0,0,...,0,0,0,0,1,1,0,1,0,1
2,3.902705,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
3,3.914741,0,1,1,0,0,1,0,1,1,...,0,0,0,0,0,1,0,1,0,1
4,3.984741,0,0,1,0,1,1,0,1,1,...,0,0,0,1,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,7.910000,1,1,0,0,0,0,1,0,0,...,1,1,0,0,1,0,0,0,1,1
668,7.940000,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
669,7.940000,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
670,7.940000,1,1,0,0,0,0,1,0,0,...,0,1,1,0,1,0,0,0,1,0


In [6]:
X, y = df.loc[:, df.columns != 'pH.preference'], df['pH.preference']
# X = df
X

,X5.FTHF_cyc.lig,AAA_25,AAA_assoc_C,AcetylCoA_hyd_C,ASH,Big_3_5,CitMHS,CpsB_CapC,CpXC,CsbD,...,SOUL,TctA,TelA,TFR_dimer,TPP_enzyme_C,TrbI,UvdE,WXXGXW,YHS,zf.CDGSH
0,1,1,1,1,0,0,1,0,0,1,...,1,0,0,0,1,1,0,1,1,1
1,1,1,1,1,0,0,1,0,0,0,...,0,0,0,0,1,1,0,1,0,1
2,1,0,0,1,0,0,1,0,0,1,...,0,0,0,0,0,1,0,1,0,0
3,0,1,1,0,0,1,0,1,1,1,...,0,0,0,0,0,1,0,1,0,1
4,0,0,1,0,1,1,0,1,1,1,...,0,0,0,1,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,1,1,0,0,0,0,1,0,0,1,...,1,1,0,0,1,0,0,0,1,1
668,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
669,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
670,1,1,0,0,0,0,1,0,0,1,...,0,1,1,0,1,0,0,0,1,0


In [7]:
# Predict the model
#pred = model_perm_subset.predict(df)
pred_perm_subset = model_perm_subset.predict(X)

# MAE Computation
#scores_MAE = mean_absolute_error(y, pred_perm_subset)
#scores_MAE = mean_absolute_error(y, pred_perm_subset)

# RMSE Computation
#scores_RMSE = np.sqrt(mean_squared_error(y, pred_perm_subset))
#print("RMSE : % f, MAE : % f" % (scores_RMSE, scores_MAE))

In [8]:
pd.DataFrame(pred_perm_subset)
pd.DataFrame(pred_perm_subset).to_csv('XXXXPredicted_pH_preferencesXXXX.csv')

## Following part is for predicting the pH for the genomes Caroline had shared with me

In [4]:
df = pd.read_csv("../data/pH_prediction/ind_pfam_gene_presence_absence_on_genome.csv", header=None)
df = df.pivot(index=0, columns=1, values=2).reset_index().rename(columns={0:"genome_id"})
df

1,genome_id,AAA_25,AAA_assoc_C,ASH,AcetylCoA_hyd_C,Big_3_5,CitMHS,CpXC,CpsB_CapC,CsbD,...,TFR_dimer,TPP_enzyme_C,TctA,TelA,TrbI,UvdE,WXXGXW,X5.FTHF_cyc.lig,YHS,zf.CDGSH
0,GCA_000005845.2_ASM584v2,1,0,0,1,1,1,1,0,1,...,0,1,0,0,0,0,0,0,0,0
1,GCA_000006765.1_ASM676v1,1,0,0,1,0,1,0,1,1,...,0,1,1,0,0,0,0,0,0,0
2,GCA_000007025.1_ASM702v1,1,0,0,1,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
3,GCA_000007205.1_ASM720v1,1,0,1,0,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
4,GCA_000007365.1_ASM736v1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,GCF_003812925.1_ASM381292v1,1,0,1,1,1,1,1,0,1,...,0,1,1,1,0,0,0,0,1,0
193,GCF_006874605.1_ASM687460v1,1,1,1,1,0,1,0,0,1,...,0,1,0,0,1,0,0,0,0,0
194,GCF_006874645.1_ASM687464v1,1,0,1,1,0,1,1,0,1,...,0,1,0,1,1,0,0,0,0,0
195,GCF_011300455.1_ASM1130045v1,1,0,1,1,0,1,0,0,0,...,0,1,0,1,0,1,0,0,1,0


In [5]:
df_input, df_genomes = df.loc[:, df.columns != 'genome_id'], df['genome_id']

In [6]:
df_input

1,AAA_25,AAA_assoc_C,ASH,AcetylCoA_hyd_C,Big_3_5,CitMHS,CpXC,CpsB_CapC,CsbD,Cys_rich_CPXG,...,TFR_dimer,TPP_enzyme_C,TctA,TelA,TrbI,UvdE,WXXGXW,X5.FTHF_cyc.lig,YHS,zf.CDGSH
0,1,0,0,1,1,1,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,1,1,1,...,0,1,1,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
3,1,0,1,0,0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,1,0,1,1,1,1,1,0,1,0,...,0,1,1,1,0,0,0,0,1,0
193,1,1,1,1,0,1,0,0,1,1,...,0,1,0,0,1,0,0,0,0,0
194,1,0,1,1,0,1,1,0,1,0,...,0,1,0,1,1,0,0,0,0,0
195,1,0,1,1,0,1,0,0,0,0,...,0,1,0,1,0,1,0,0,1,0


In [7]:
Caroline_genomes_pH_pred = model_perm_subset.predict(df_input)
Caroline_genomes_pH_pred

array([6.7027493, 5.49116  , 5.575839 , 5.4847198, 5.6631346, 5.975887 ,
       6.2520814, 4.690501 , 5.666716 , 5.7117996, 5.5461907, 5.640215 ,
       5.9255905, 5.940655 , 6.194208 , 6.2384915, 5.338949 , 4.9721665,
       5.788143 , 6.388763 , 5.560687 , 5.453991 , 5.992248 , 5.948341 ,
       5.2857375, 5.8696384, 5.9918523, 6.151775 , 5.508456 , 5.4360733,
       6.120976 , 6.215288 , 5.9773464, 5.2041616, 5.998891 , 5.5983067,
       5.5675178, 6.2506065, 6.0877686, 4.866733 , 6.321186 , 6.1052217,
       6.049795 , 5.76063  , 5.5191736, 6.3638825, 6.3658957, 5.7110896,
       5.431964 , 5.8429914, 6.842036 , 6.2149134, 5.7542324, 6.0428486,
       5.5760217, 5.808072 , 6.260253 , 6.239827 , 6.7956376, 6.6290665,
       6.01328  , 6.1917276, 5.745524 , 6.916538 , 5.9211345, 6.310611 ,
       6.359042 , 6.1036015, 5.8767953, 5.526765 , 5.3217177, 6.7110567,
       6.65433  , 5.295489 , 5.414089 , 5.9549003, 6.2365885, 6.266329 ,
       5.5496683, 5.727381 , 5.718588 , 4.7880774, 

In [8]:
df['pH_pred'] = Caroline_genomes_pH_pred
df

1,genome_id,AAA_25,AAA_assoc_C,ASH,AcetylCoA_hyd_C,Big_3_5,CitMHS,CpXC,CpsB_CapC,CsbD,...,TPP_enzyme_C,TctA,TelA,TrbI,UvdE,WXXGXW,X5.FTHF_cyc.lig,YHS,zf.CDGSH,pH_pred
0,GCA_000005845.2_ASM584v2,1,0,0,1,1,1,1,0,1,...,1,0,0,0,0,0,0,0,0,6.702749
1,GCA_000006765.1_ASM676v1,1,0,0,1,0,1,0,1,1,...,1,1,0,0,0,0,0,0,0,5.491160
2,GCA_000007025.1_ASM702v1,1,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,5.575839
3,GCA_000007205.1_ASM720v1,1,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,5.484720
4,GCA_000007365.1_ASM736v1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,5.663135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,GCF_003812925.1_ASM381292v1,1,0,1,1,1,1,1,0,1,...,1,1,1,0,0,0,0,1,0,6.284989
193,GCF_006874605.1_ASM687460v1,1,1,1,1,0,1,0,0,1,...,1,0,0,1,0,0,0,0,0,6.053369
194,GCF_006874645.1_ASM687464v1,1,0,1,1,0,1,1,0,1,...,1,0,1,1,0,0,0,0,0,5.384559
195,GCF_011300455.1_ASM1130045v1,1,0,1,1,0,1,0,0,0,...,1,0,1,0,1,0,0,1,0,5.201993


In [10]:
df.to_csv("../data/pH_prediction/Caroline_genomes_pH_predictions.csv")

## This part is for the prediction of the species tree entries

In [3]:
species_df = pd.read_csv("../data/species_tree/pH_prediction/ind_pfam_gene_presence_absence_on_genome.csv", header=None)
species_df = species_df.pivot(index=0, columns=1, values=2).reset_index().rename(columns={0:"genome_id"})
species_df

1,genome_id,AAA_25,AAA_assoc_C,ASH,AcetylCoA_hyd_C,Big_3_5,CitMHS,CpXC,CpsB_CapC,CsbD,...,TFR_dimer,TPP_enzyme_C,TctA,TelA,TrbI,UvdE,WXXGXW,X5.FTHF_cyc.lig,YHS,zf.CDGSH
0,GCA_000005845.2_ASM584v2,1,0,0,1,1,1,1,0,1,...,0,1,0,0,0,0,0,0,0,0
1,GCA_000006765.1_ASM676v1,1,0,0,1,0,1,0,1,1,...,0,1,1,0,0,0,0,0,0,0
2,GCA_000007025.1_ASM702v1,1,0,0,1,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
3,GCA_000007205.1_ASM720v1,1,0,1,0,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
4,GCA_000007365.1_ASM736v1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,GCA_900169565.1_NSJP_Ch1,1,0,0,0,0,1,1,0,1,...,1,1,0,0,0,0,0,0,1,1
179,GCA_900184705.1_CAMBI-1,1,0,0,1,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
180,GCA_900187105.1_50618_F01,1,0,0,0,0,1,0,1,1,...,0,1,0,0,0,0,0,0,0,0
181,GCA_900474605.1_41594_C01,1,1,0,0,0,1,1,1,0,...,1,1,0,1,0,0,0,0,0,0


In [4]:
species_df_input, species_df_genomes = species_df.loc[:, species_df.columns != 'genome_id'], species_df['genome_id']

In [7]:
species_pH_pred = model_perm_subset.predict(species_df_input)
species_pH_pred

array([6.7027493, 5.49116  , 5.575839 , 5.4847198, 5.6631346, 5.975887 ,
       6.2520814, 4.690501 , 5.666716 , 5.7117996, 5.5461907, 5.640215 ,
       5.9255905, 5.940655 , 6.194208 , 6.2384915, 5.338949 , 4.9721665,
       5.788143 , 6.388763 , 5.560687 , 5.453991 , 5.992248 , 5.948341 ,
       5.2857375, 5.8696384, 5.9918523, 6.151775 , 5.508456 , 5.4360733,
       6.120976 , 6.215288 , 5.9773464, 5.2041616, 5.998891 , 5.5983067,
       5.5675178, 6.2506065, 6.0877686, 4.866733 , 6.321186 , 6.1052217,
       6.049795 , 5.76063  , 5.5191736, 6.3638825, 6.3658957, 5.7110896,
       5.431964 , 5.8429914, 6.842036 , 6.2149134, 5.7542324, 6.0428486,
       5.5760217, 5.808072 , 6.260253 , 6.239827 , 6.7956376, 6.6290665,
       6.01328  , 6.1917276, 5.745524 , 6.916538 , 5.9211345, 6.310611 ,
       6.359042 , 6.1036015, 5.8767953, 5.526765 , 5.3217177, 6.7110567,
       6.65433  , 5.295489 , 5.414089 , 5.9549003, 6.2365885, 6.266329 ,
       5.5496683, 5.727381 , 5.718588 , 4.7880774, 

In [8]:
species_df['pH_pred'] = species_pH_pred
species_df

1,genome_id,AAA_25,AAA_assoc_C,ASH,AcetylCoA_hyd_C,Big_3_5,CitMHS,CpXC,CpsB_CapC,CsbD,...,TPP_enzyme_C,TctA,TelA,TrbI,UvdE,WXXGXW,X5.FTHF_cyc.lig,YHS,zf.CDGSH,pH_pred
0,GCA_000005845.2_ASM584v2,1,0,0,1,1,1,1,0,1,...,1,0,0,0,0,0,0,0,0,6.702749
1,GCA_000006765.1_ASM676v1,1,0,0,1,0,1,0,1,1,...,1,1,0,0,0,0,0,0,0,5.491160
2,GCA_000007025.1_ASM702v1,1,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,5.575839
3,GCA_000007205.1_ASM720v1,1,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,5.484720
4,GCA_000007365.1_ASM736v1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,5.663135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,GCA_900169565.1_NSJP_Ch1,1,0,0,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,1,5.741438
179,GCA_900184705.1_CAMBI-1,1,0,0,1,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,6.118430
180,GCA_900187105.1_50618_F01,1,0,0,0,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,5.749992
181,GCA_900474605.1_41594_C01,1,1,0,0,0,1,1,1,0,...,1,0,1,0,0,0,0,0,0,5.872581


In [9]:
species_df.to_csv("../data/species_tree/pH_prediction/species_tree_pH_predictions.csv", index=False)